In [3]:
import json
from datetime import datetime

import stix2 as stix
from jupyter_widget_stixview.widget import StixviewGraph as Graph

In [4]:
identity_mustermann = stix.Identity(
    created=datetime.strptime("01.01.1970", "%d.%m.%Y"),
    identity_class=stix.v21.vocab.IDENTITY_CLASS_INDIVIDUAL,
    name="Max Mustermann",
    contact_information="info@nextpart.io",
    labels=["person"],
    confidence=100,
)
identity_nxtp = stix.Identity(
    created=datetime.strptime("16.09.2019", "%d.%m.%Y"),
    name="Nextpart Security Intelligence GmbH",
    sectors=stix.v21.vocab.INDUSTRY_SECTOR_TECHNOLOGY,
    contact_information="info@nextpart.io",
    labels=["company"],
    lang="German",
    confidence=100,
    created_by_ref=identity_mustermann.id,
)
location_nxtp_hq = stix.Location(
    created_by_ref=identity_mustermann.id,
    name="Address: Schmidtorstraße 4/2, 4020 Linz, AT",
    description="Schmidtorstraße 4/2, 4020 Linz, AT",
    labels=["address", "company", "primary"],
    region=stix.v21.vocab.REGION_EUROPE,
    country="Austria",
    city="Linz",
    postal_code="4020",
    street_address="Schmidtorstraße 4/2",
    longitude=14.287532285406582,
    latitude=48.30463730941664,
    confidence=100,
    external_references=[
        stix.ExternalReference(
            source_name="Firmenbucheintrag",
            description="Firmenbuch is an open book where austrian companies are listed.",
        )
    ],
)
relation_nxtp_located_hq = stix.Relationship(
    relationship_type="located-at",
    source_ref=identity_nxtp,
    target_ref=location_nxtp_hq,
    start_time=datetime.strptime("05.09.2020", "%d.%m.%Y"),
    created_by_ref=identity_mustermann.id,
)
location_austria = stix.Location(
    created_by_ref=identity_mustermann.id,
    name="Country: AT",
    description="country",
    region=stix.v21.vocab.REGION_EUROPE,
    country="Austria",
    labels=["country"],
    confidence=100,
)
relation_aut_derived_nxtp_hq = stix.Relationship(
    relationship_type="derived-from",
    source_ref=location_austria,
    target_ref=location_nxtp_hq,
    start_time=datetime.strptime("05.09.2020", "%d.%m.%Y"),
    created_by_ref=identity_mustermann.id,
)
relation_mustermann_located_aut = stix.Relationship(
    relationship_type="located-at",
    source_ref=identity_mustermann,
    target_ref=location_austria,
    start_time=datetime.strptime("05.09.2020", "%d.%m.%Y"),
    created_by_ref=identity_mustermann.id,
)
bundle_mustermann_baseinfo = stix.Bundle(
    objects=[
        identity_nxtp,
        identity_mustermann,
        location_nxtp_hq,
        location_austria,
        relation_nxtp_located_hq,
        relation_aut_derived_nxtp_hq,
        relation_mustermann_located_aut,
    ],
)
latest_bundle = bundle_mustermann_baseinfo
Graph(bundle=json.loads(bundle_mustermann_baseinfo.serialize()))

StixviewGraph(bundle={'type': 'bundle', 'id': 'bundle--ca1782a2-6a6d-42de-8b67-18eae9110bcf', 'objects': [{'ty…

In [5]:

user_acc_mustermann = stix.UserAccount(
    display_name="Max Mustermann",
    user_id="mustermann@nextpart.io",
    is_disabled=False,
    # is_service_account=False,
    # is_privileged=False,
    # can_escalate_privs=True,
    account_type=stix.v21.vocab.ACCOUNT_TYPE_OPENID,
    account_created=datetime.strptime("01.01.1970", "%d.%m.%Y"),
)
email_addr_mustermann = stix.EmailAddress(
    value="mustermann@nextpart.io",
    display_name="Max Mustermann",
    belongs_to_ref=user_acc_mustermann.id,
)
email_addr_nxtp = stix.EmailAddress(
    value="info@nextpart.io", display_name="Nextpart Security Intelligence GmbH"
)
relation_email_addr_derived_mustermann = stix.Relationship(
    relationship_type="derived-from",
    source_ref=email_addr_mustermann,
    target_ref=identity_mustermann,
    created_by_ref=identity_mustermann.id,
)
relation_email_addr_derived_nxtp = stix.Relationship(
    relationship_type="derived-from",
    source_ref=email_addr_nxtp.id,
    target_ref=identity_nxtp.id,
    created_by_ref=identity_mustermann.id,
)
relation_acc_related_mustermann = stix.Relationship(
    relationship_type="related-to",
    source_ref=user_acc_mustermann,
    target_ref=identity_mustermann,
    created_by_ref=identity_mustermann.id,
)
relation_mustermann_related_nxtp = stix.Relationship(
    relationship_type="related-to",
    source_ref=identity_mustermann.id,
    target_ref=identity_nxtp.id,
    created_by_ref=identity_mustermann.id,
)

bundle_mustermann_enhancedinfo = stix.Bundle(
    objects=(
        bundle_mustermann_baseinfo.objects
        + [
            user_acc_mustermann,
            email_addr_mustermann,
            email_addr_nxtp,
            relation_email_addr_derived_mustermann,
            relation_email_addr_derived_nxtp,
            relation_acc_related_mustermann,
            relation_mustermann_related_nxtp,
        ]
    )
)
latest_bundle = bundle_mustermann_enhancedinfo
Graph(bundle=json.loads(bundle_mustermann_enhancedinfo.serialize()))

StixviewGraph(bundle={'type': 'bundle', 'id': 'bundle--f877b600-e7f8-4250-b8c4-e04898ca64c6', 'objects': [{'ty…

In [6]:

domain_nxtp = stix.DomainName(value="nextpart.io", defanged=False)
infra_nxtp = stix.Infrastructure(
    name="Nextpart",
    aliases="Nextpart Security Intellgience GmbH",
    labels=["company"],
    created_by_ref=identity_nxtp.id,
)
relation_infra_consists_domain = stix.Relationship(
    relationship_type="consists-of", source_ref=infra_nxtp.id, target_ref=domain_nxtp.id
)
relation_infra_consists_email_addr_mustermann = stix.Relationship(
    relationship_type="consists-of",
    source_ref=infra_nxtp.id,
    target_ref=email_addr_mustermann.id,
)
relation_infra_consists_email_addr_nxtp = stix.Relationship(
    relationship_type="consists-of",
    source_ref=infra_nxtp.id,
    target_ref=email_addr_nxtp.id,
)
relation_infra_consists_user_acc = stix.Relationship(
    relationship_type="consists-of",
    source_ref=infra_nxtp.id,
    target_ref=user_acc_mustermann.id,
)
relation_infra_located_at = stix.Relationship(
    relationship_type="located-at",
    source_ref=infra_nxtp.id,
    target_ref=location_austria.id,
)
relation_infra_relates_nxtp = stix.Relationship(
    relationship_type="related-to",
    source_ref=infra_nxtp.id,
    target_ref=identity_nxtp.id,
)
relation_domain_derived_email_addr_nxtp = stix.Relationship(
    relationship_type="derived-from",
    source_ref=domain_nxtp.id,
    target_ref=email_addr_nxtp.id,
)
bundle_nxtp_enhancedinfo = stix.Bundle(
    objects=bundle_mustermann_enhancedinfo.objects
    + [
        domain_nxtp,
        infra_nxtp,
        relation_infra_consists_domain,
        relation_infra_consists_email_addr_mustermann,
        relation_infra_consists_email_addr_nxtp,
        relation_infra_consists_user_acc,
        relation_infra_located_at,
        relation_infra_relates_nxtp,
        relation_domain_derived_email_addr_nxtp
    ],
)
latest_bundle = bundle_nxtp_enhancedinfo
Graph(bundle=json.loads(bundle_nxtp_enhancedinfo.serialize()))

StixviewGraph(bundle={'type': 'bundle', 'id': 'bundle--99e18a4a-19eb-4b7b-b4ba-501d0bee6953', 'objects': [{'ty…

In [7]:

url_investigation_input = stix.URL(
    value="https://satyammould.com/d.php?redacted",
    defanged=False,
)
observed_data_investigation_input = stix.ObservedData(
    first_observed=datetime.strptime("11.08.2021 20:21:10", "%d.%m.%Y %H:%M:%S"),
    last_observed=datetime.strptime("11.08.2021 20:21:10", "%d.%m.%Y %H:%M:%S"),
    number_observed=1,
    object_refs=[url_investigation_input.id],
)

grouping_url_investigation = stix.Grouping(
    name="Indicator Enrichment: https://satyammould[.]com/d.php?redacted",
    context=stix.v21.vocab.GROUPING_CONTEXT_UNSPECIFIED,
    created_by_ref=identity_mustermann.id,
    object_refs=[observed_data_investigation_input.id, url_investigation_input.id],
)

bundle_url_investigation_base = stix.Bundle(
    objects=[
        url_investigation_input,
        observed_data_investigation_input,
        grouping_url_investigation,
    ]
)
Graph(bundle=json.loads(bundle_url_investigation_base.serialize()))

StixviewGraph(bundle={'type': 'bundle', 'id': 'bundle--ac305651-f751-4b38-b761-2faab43a4b5c', 'objects': [{'ty…

In [8]:

identity_urlscan = stix.Identity(
    name="urlscan GmbH",
    description="urlscan.io is a free service to scan and analyse websites",
    identity_class=stix.v21.vocab.IDENTITY_CLASS_ORGANIZATION,
    sectors=[stix.v21.vocab.INDUSTRY_SECTOR_TECHNOLOGY],
    contact_information="E-Mail: info@urlscan.io;Telefon: +49 241 430 38 28;Web: urlscan.io",
)
asn_investigation = stix.AutonomousSystem(number=13335)
ipv6_urlscan_result = stix.IPv4Address(
    value="2606:4700:3034::6815:28bb",
    defanged=False,
)
domain_investigation_input = stix.DomainName(
    value="satyammould.com", defanged=False, resolves_to_refs=ipv6_urlscan_result.id
)
relation_domain_derived_url = stix.Relationship(
    relationship_type="derived-from",
    source_ref=domain_investigation_input.id,
    target_ref=url_investigation_input.id,
    created_by_ref=identity_urlscan.id,
)
certificate_investigation_domain = stix.X509Certificate(
    subject="sni.cloudflaressl.com",
    validity_not_before=datetime.fromtimestamp(1621209600),
    validity_not_after=datetime.fromtimestamp(1652745599),
    signature_algorithm="AES-128-GCM",
    issuer="Cloudflare Inc ECC CA-3",
)
relation_domain_related_cert = stix.Relationship(
    relationship_type="related-to",
    source_ref=certificate_investigation_domain.id,
    target_ref=domain_investigation_input.id,
    created_by_ref=identity_urlscan.id,
)

infra_investigation = stix.Infrastructure(
    name="satyammould",
    created_by_ref=identity_urlscan.id,
)
infra_cloudflair = stix.Infrastructure(
    name="Cloudflair",
    created_by_ref=identity_urlscan.id,
)
relation_case_use_cloudflair = stix.Relationship(
    relationship_type="uses",
    source_ref=infra_investigation.id,
    target_ref=infra_cloudflair.id,
    created_by_ref=identity_urlscan.id,
)


report_urlscan = stix.Report(
    name="URLscan.io Report for https://satyammould[.]com/d.php?redacted",
    report_types=[
        stix.v21.vocab.REPORT_TYPE_OBSERVED_DATA,
        stix.v21.vocab.REPORT_TYPE_TOOL,
    ],
    published=datetime.strptime("11.08.2021 20:21:11", "%d.%m.%Y %H:%M:%S"),
    object_refs=[
        certificate_investigation_domain,
        domain_investigation_input,
        domain_investigation_input.id,
        infra_cloudflair,
        infra_investigation,
        ipv6_urlscan_result.id,
        relation_case_use_cloudflair,
        relation_domain_derived_url,
        relation_domain_related_cert,
        url_investigation_input.id,
        asn_investigation,
    ],
    created_by_ref=identity_urlscan.id,
)
bundle_url_investigation_urlscan_base = stix.Bundle(
    objects=bundle_url_investigation_base.objects
    +
    [
        certificate_investigation_domain,
        domain_investigation_input,
        identity_urlscan,
        infra_cloudflair,
        infra_investigation,
        ipv6_urlscan_result,
        relation_case_use_cloudflair,
        relation_domain_derived_url,
        relation_domain_related_cert,
        report_urlscan,
        asn_investigation,
    ]
)
Graph(bundle=json.loads(bundle_url_investigation_urlscan_base.serialize()))

StixviewGraph(bundle={'type': 'bundle', 'id': 'bundle--bb009a94-44a8-41f5-89a7-1cf20514c1ce', 'objects': [{'ty…